Цель:


Скрипт, строящий vowpal wabbit модели для классификации текстов /n
И подготавливающий результаты для сабмита в


https://www.kaggle.com/c/made-ml-2019-hw1/overview


Дата:


29.10.2019

# Библиотеки

In [1]:
from vowpalwabbit import pyvw
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import accuracy_score, log_loss
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from random import randrange
import numpy as np
from gensim.parsing import preprocessing as prep
from collections import defaultdict
import pandas as pd

# Импорт данных

In [2]:
df_train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header = None)
df_train.columns = ["text", "assessment"]
df_train.head()

,text,assessment
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [3]:
# Уберем слова, которые встречаются реже, чем 2 раза
class FilterRareWords(object):
    def __init__(self):
        self.cv = defaultdict(int)
    def fit(self, texts):
        for text in texts:
            for word in text.split():
                self.cv[word] += 1
    def __call__(self, text):
        return ' '.join([self.filter_word(word) for word in text.split()])
    def filter_word(self, word):
        return '' if self.cv[word] < 2 else word

In [4]:
filter_words = FilterRareWords()

In [5]:
data_train, y_train = df_train.text, df_train.assessment

In [6]:
filter_words.fit(data_train)
data_train = data_train.apply(filter_words)

In [7]:
train, valid, train_labels, valid_labels  =   train_test_split(data_train, y_train, test_size=0.2, shuffle=True)

# Подготовка данных для vw

In [8]:
def to_vw_format(document, label=None):
#     Функция, переводящая данные в формат пригодный для vw
#     document - текст
#     label - target
    num_w=len(document.split(" "))
    delete_space = re.compile("[.;:!\',\"()\[\]]")
    return str(label or '') + ' |text ' + " ".join([ word for word in delete_space.sub("",  document.lower()).split(" ")]) +' |meta ' + 'num_sym:'+str(len(document))+' num_words:'+str(num_w)+' '+'\n'

In [9]:
training_samples = []
for text, target in zip(train, train_labels):
    training_samples.append(to_vw_format(str(text), 1 if target == 1 else -1)[:-2])
    
test_samples = []
for text, target in zip(valid, valid_labels):
    test_samples.append(to_vw_format(str(text), 1 if target == 1 else -1)[:-2])

# Моделька

Игры с hyperopt

In [10]:
def hyperopt_vw(params):
# Функция, для оценки качества модели в хиперопте
    scores=[]
    folds_quant=5
    msk= [randrange(folds_quant) for p in range(len(training_samples))] 

    for i in range(folds_quant):
#           Ручками посчитаем кроссвалидацию на folds_quant фолдов
        model = pyvw.vw(
            **params
        )
        train=np.array(training_samples)[list(np.array(msk) != i)]
        test=np.array(training_samples)[list(np.array(msk) == i)]
        test_labels=np.array(train_labels)[list(np.array(msk) == i)]

        for iteration in range(25):
            for k in range(len(train)):
                model.learn(train[k])
        
        test_predictions = [model.predict(sample) for sample in test]
  
#        print(test_predictions)
        scores.append(log_loss(test_labels, test_predictions))
        
        model.finish()

    print(params)
    print(np.array(scores).mean())
    return  np.array(scores).mean()

param_grid = { 
"ngram" : hp.choice('ngram', range(1, 3)),
"skips" : hp.choice('skips', range(1, 3)),
"l1" :  hp.quniform('l1', 0.000001, 0.00003, 0.000001),
"l2" : hp.quniform('l2', 0.000001, 0.00003, 0.000001),
"b" : hp.choice('b', range(22, 27)),
"learning_rate" : 0.3,
"link": 'logistic',
"loss_function" : 'logistic'
# "bootstrap" : hp.choice('bootstrap', range(0, 40,10))
# "hash": 'all'
}
                  
best_params = fmin(fn=hyperopt_vw, space=param_grid, algo=tpe.suggest, max_evals=10,verbose=0)


{'b': 26, 'l1': 2e-06, 'l2': 1.2e-05, 'learning_rate': 0.3, 'link': 'logistic', 'loss_function': 'logistic', 'ngram': 2, 'skips': 2}
0.4774645274720511                                                                                                     
{'b': 24, 'l1': 2.6e-05, 'l2': 1.4999999999999999e-05, 'learning_rate': 0.3, 'link': 'logistic', 'loss_function': 'logistic', 'ngram': 2, 'skips': 2}
0.47809636315658627                                                                                                    
{'b': 23, 'l1': 1.3e-05, 'l2': 3e-06, 'learning_rate': 0.3, 'link': 'logistic', 'loss_function': 'logistic', 'ngram': 2, 'skips': 2}
0.47071889722688437                                                                                                    
{'b': 22, 'l1': 2.6e-05, 'l2': 6e-06, 'learning_rate': 0.3, 'link': 'logistic', 'loss_function': 'logistic', 'ngram': 1, 'skips': 2}
0.4786642388354557                                                                         

In [11]:
best_params

{'b': 1, 'l1': 4.9999999999999996e-06, 'l2': 7e-06, 'ngram': 1, 'skips': 0}

Важно заметить, что hp.choice возвращает не лучшее значение, а его id

# Обучение и праверка на валидационой выборке

In [42]:

vw = pyvw.vw( 
    loss_function='logistic',

    link='logistic',
    b=22,
#     bootstrap=20,
    q='aa',
#     cubic='aaa',
    ngram=2,
#     skips=1,
#     hash='all',
#     hessian_on=True,
#     random_seed=112,
    l1=0.00001,
    l2=0.00001,
    f='vw.log.model',
    learning_rate=0.4)

for iteration in range(50):
    if iteration % 10 ==0:
        train_predictions = [vw.predict(sample) for sample in training_samples]
        test_predictions = [vw.predict(sample) for sample in test_samples]
        print("========== step {0} ==========".format(iteration))
        print("train LogLoss: ",log_loss(train_labels, train_predictions))
        print("test LogLoss: ",log_loss(valid_labels, test_predictions))
    for i in range(len(training_samples)):
            vw.learn(training_samples[i])
vw.finish()


========== step 0 ==========
train LogLoss:  0.6931471805599452
test LogLoss:  0.6931471805599452
========== step 10 ==========
train LogLoss:  0.1072883712939382
test LogLoss:  0.46118086761387417
========== step 20 ==========
train LogLoss:  0.08538240442453962
test LogLoss:  0.4568272036228802
========== step 30 ==========
train LogLoss:  0.07803716815071017
test LogLoss:  0.45516854161199943
========== step 40 ==========
train LogLoss:  0.07457112156337303
test LogLoss:  0.4548718598721998


Более подробно можно посмотреть в 
https://www.analyticsvidhya.com/blog/2018/01/online-learning-guide-text-classification-vowpal-wabbit-vw/ https://github.com/VowpalWabbit/vowpal_wabbit/blob/master/python/examples/poisson_regression.ipynb https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Command-line-arguments

# Обучимся всей выборке

Идея - модель не стабильная  так как мало наблюдений дававйте повысим стабильность убычившись на кроссвалидации и усреднив результат

In [13]:
training_full = []
for text, target in zip(data_train, y_train):
    training_full.append(to_vw_format(str(text), 1 if target == 1 else -1)[:-2])

In [14]:
# Количество фолдов для обучения
folds_quant=5

msk= [randrange(folds_quant) for p in range(len(training_full))] 
model=[]

for i in range(folds_quant):

    vw = pyvw.vw( 
        loss_function='logistic',
        link='logistic',
        b=22,
        q='aa',
        cubic='aaa',
        ngram=2,
        skips=1,
        l1=0.00001,
        l2=0.00001,
        f='vw.log.model',
        learning_rate=0.4)

    model.append(vw)

    train=np.array(training_full)[list(np.array(msk) != i)]
    test=np.array(training_full)[list(np.array(msk) == i)]
    test_labels=np.array(y_train)[list(np.array(msk) == i)]

    for iteration in range(25):
        if iteration % 5 ==0:
            train_predictions = [vw.predict(sample) for sample in training_full]
            print("========== step {0} itteration {1} ==========".format(iteration,i))
            print("train LogLoss: ",log_loss(y_train, train_predictions))
        for k in range(len(train)):
            model[i].learn(train[k])

    print("\n")

========== step 0 itteration 0 ==========
train LogLoss:  0.6931471805599454
========== step 5 itteration 0 ==========
train LogLoss:  0.18797120127096517
========== step 10 itteration 0 ==========
train LogLoss:  0.16518528449395437
========== step 15 itteration 0 ==========
train LogLoss:  0.15738978457719702
========== step 20 itteration 0 ==========
train LogLoss:  0.15376119966943783


========== step 0 itteration 1 ==========
train LogLoss:  0.6931471805599454
========== step 5 itteration 1 ==========
train LogLoss:  0.18873199218674733
========== step 10 itteration 1 ==========
train LogLoss:  0.16468647139476117
========== step 15 itteration 1 ==========
train LogLoss:  0.1563548793178801
========== step 20 itteration 1 ==========
train LogLoss:  0.15240385181440652


========== step 0 itteration 2 ==========
train LogLoss:  0.6931471805599454
========== step 5 itteration 2 ==========
train LogLoss:  0.18942915627751747
========== step 10 itteration 2 ==========
train LogLoss: 

# Скорим тестовую выборку

In [15]:
df_test = pd.read_csv('products_sentiment_test.tsv', sep='\t', header = None,skiprows=1)
df_test.columns = ["id", "text"]

In [16]:
test_text= df_test.text

In [17]:
test_text = test_text.apply(filter_words)

In [19]:
test = []
for text in test_text:
    test.append(to_vw_format(str(text), 1)[:-2])

In [20]:
test_out=[]
for i in range(len(model)):
    test_out.append([model[i].predict(sample) for sample in test])

Усредняем прогнозы 5 лучших моделей

In [23]:
results=pd.DataFrame()
results['id']=df_test.id
results['y']=(np.array(test_out).mean(axis=0))

In [24]:
results

,id,y
0,0,0.950885
1,1,0.258813
2,2,0.735756
3,3,0.685699
4,4,0.301737
...,...,...
495,495,0.514314
496,496,0.625902
497,497,0.113846
498,498,0.505971


In [25]:
results.to_csv('results.csv',index=False)